In [16]:
import glob
import os
import numpy as np
import pandas as pd
import sys

from collections import OrderedDict

from matplotlib.patches import Rectangle
from matplotlib import pyplot as plt
# %matplotlib qt5

from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
from scipy.stats import f_oneway
from scipy.stats import normaltest



import config 
sys.path.append('./models/v1model/')
from plotting import _get_barplot
from utils import turn_tex
turn_tex('on')

path = '/run/media/loaloa/lbb_ssd/primitives/processed2/results/'
%matplotlib qt5

In [17]:

data = pd.read_csv(f'{path}/all_primitives_results_new.csv', index_col=[0,1,2])
data

0         1         2         3  \
exp_name     DIV design                                                    
Cue gradient 14  480um x 12/12   -0.126280 -0.126280  0.075472 -0.406302   
                 480um x 12/11   -1.000000 -0.117216 -0.347942 -1.000000   
                 480um x 12/10   -1.000000  0.152225 -0.191854 -0.191854   
                 480um x 12/9    -0.174825 -0.161657 -0.161657 -0.161657   
                 480um x 12/8    -0.826362 -0.826362 -0.038380 -0.038380   
...                                    ...       ...       ...       ...   
Growth speed 7   W:10um - H:24um       NaN       NaN  0.031330  0.038363   
                 W:14um - H:24um  0.480818       NaN  0.514066  0.064578   
                 W:18um - H:24um  0.322890       NaN  0.023657  0.075448   
                 W:22um - H:24um       NaN  0.631713       NaN  0.054987   
                 W:26um - H:24um  0.086956  0.146419  0.111892  0.407928   

                                         4         5         6         7  \
exp_name     DIV design                                                    
Cue gradient 14  480um x 12/12   -0.406302 -1.000000 -0.815404 -0.815404   
                 480um x 12/11    1.000000       NaN       NaN       NaN   
                 480um x 12/10   -0.191854 -1.000000 -1.000000 -0.461538   
                 480um x 12/9    -0.161657  0.340446 -1.000000 -1.000000   
                 480um x 12/8    -0.038380 -0.038380 -0.384030 -0.384030   
...                                    ...       ...       ...       ...   
Growth speed 7   W:10um - H:24um       NaN       NaN       NaN  0.104220   
                 W:14um - H:24um       NaN       NaN  0.079284  0.251279   
                 W:18um - H:24um       NaN       NaN  0.263427  0.312020   
                 W:22um - H:24um       NaN       NaN       NaN  0.340153   
                 W:26um - H:24um       NaN       NaN       NaN  0.206522   

                                         8         9  ...  36  37  38  39  40  \
exp_name     DIV design                               ...                       
Cue gradient 14  480um x 12/12   -0.815404 -0.815404  ... NaN NaN NaN NaN NaN   
                 480um x 12/11         NaN       NaN  ... NaN NaN NaN NaN NaN   
                 480um x 12/10         NaN       NaN  ... NaN NaN NaN NaN NaN   
                 480um x 12/9          NaN       NaN  ... NaN NaN NaN NaN NaN   
                 480um x 12/8     0.457060  0.457060  ... NaN NaN NaN NaN NaN   
...                                    ...       ...  ...  ..  ..  ..  ..  ..   
Growth speed 7   W:10um - H:24um  0.104220       NaN  ... NaN NaN NaN NaN NaN   
                 W:14um - H:24um  0.251279       NaN  ... NaN NaN NaN NaN NaN   
                 W:18um - H:24um  0.312020       NaN  ... NaN NaN NaN NaN NaN   
                 W:22um - H:24um  0.340153       NaN  ... NaN NaN NaN NaN NaN   
                 W:26um - H:24um  0.206522       NaN  ... NaN NaN NaN NaN NaN   

                                  41  42  43  44  45  
exp_name     DIV design                               
Cue gradient 14  480um x 12/12   NaN NaN NaN NaN NaN  
                 480um x 12/11   NaN NaN NaN NaN NaN  
                 480um x 12/10   NaN NaN NaN NaN NaN  
                 480um x 12/9    NaN NaN NaN NaN NaN  
                 480um x 12/8    NaN NaN NaN NaN NaN  
...                               ..  ..  ..  ..  ..  
Growth speed 7   W:10um - H:24um NaN NaN NaN NaN NaN  
                 W:14um - H:24um NaN NaN NaN NaN NaN  
                 W:18um - H:24um NaN NaN NaN NaN NaN  
                 W:22um - H:24um NaN NaN NaN NaN NaN  
                 W:26um - H:24um NaN NaN NaN NaN NaN  

[154 rows x 46 columns]

In [18]:
def compute_singles(group_values, group_levels):
    mwu_pvalues = []
    for x, x_name in zip(group_values, group_levels):
        mwu_pvals = []
        for y, y_name in zip(group_values, group_levels):
            p = mannwhitneyu(x, y).pvalue
            mwu_pvals.append(p)
            print(f'{x_name}, {y_name}: p: {p:.4f}')
        mwu_pvalues.append(mwu_pvals)
    mwu_pvalues = np.array(mwu_pvalues)

    fig, ax = plt.subplots()
    fig.subplots_adjust(bottom=.2)
    im = ax.imshow(mwu_pvalues, vmin=0, vmax=.3)
    plt.scatter(*np.where(mwu_pvalues<0.05), marker='*', color='w', s=100)
    plt.colorbar(im, orientation='vertical', label='p-value MWU test')
    plt.title(exp_name)

    lbls = [l[0] for l in group_levels]
    ax.set_yticklabels((lbls))
    ax.set_yticks(np.arange(len(lbls)))
    ax.set_xticklabels((lbls), rotation=45, ha='right')
    ax.set_xticks(np.arange(len(lbls)))



def test_kruskal_wallis(exp_data, factor, parametric=False, passed_designs=None):
    
    designs = [[d] for d in exp_data.index.unique('design')]
    if passed_designs is not None:
        designs = passed_designs
    
    groups = [exp_data.loc[d].stack().dropna() for d in designs if len(exp_data.loc[d].stack().dropna()) >2]
    group_levels = [g.name for g in groups]
    group_values = [g.values for g in groups]

    if not parametric:
        t_stat, p = kruskal(*group_values)
        which_test = 'Kruskal Wallis'
    else:
        t_stat, p = f_oneway(*group_values)
        which_test = 'ANOVA'
        var_homog = sorted([np.var(gv) for gv in group_values])
        var_homog = max(var_homog)/min(var_homog)
        normal = [normaltest(gv).pvalue if len(gv) > 7 else 0 for gv in group_values]
        normal = [p>.05 if p != 0 else 'too few samples' for p in normal]
    
    sig = '***' if p <.05 else 'N.S.'
    lbl = f'{config.SPACER}\n{factor}: n levels:{len(designs)}:\n {which_test} t={t_stat:.3f}, p={p:.4f}\n --> {sig}\n\n'
    if parametric:
        print(f'Within group normal distr: {normal}')
        print(f'Variance homogeneity (max(group_var)/min(group_var)): {var_homog}')


    print(lbl)
    
    # if passed_designs is None:
    compute_singles(group_values, designs)
    
            
exp_names = data.index.unique('exp_name')
print(exp_names)
div = 'merge'
for exp_name in exp_names:
    if div in (7,14):
        exp_dat = data.loc[(exp_name, div),:]
    elif div in ('split', 'merge'):
        exp_dat = data.loc[exp_name,:]
        if div == 'merge':
            exp_dat = exp_dat.unstack('DIV').T.reset_index(drop=True).T
    exp_name = f'{exp_name} DIV:{div}'
    print(exp_name)


    designs = None
    if exp_name.startswith('Edge transition'):
        exp_name += '_radius'
        designs = (
        ['2um, 15um','3.5um, 15um','5.5um, 15um',],
        ['2um, 25um','3.5um, 25um','5.5um, 25um',],
        ['2um, inf','3.5um, inf','5.5um, inf',], 
        ['neg. control'],)
        p = test_kruskal_wallis(exp_dat, exp_name, passed_designs=designs)
        
        exp_name += '_opening'
        designs = (
        ['5.5um, 15um','5.5um, 25um','5.5um, inf',], 
        ['3.5um, 15um','3.5um, 25um','3.5um, inf',],
        ['2um, 15um','2um, 25um','2um, inf',],
        ['neg. control'],)
        p = test_kruskal_wallis(exp_dat, exp_name, passed_designs=designs)
        
    elif exp_name.startswith('Growth speed'):
        exp_name += '_chnl_height_comparison'

        designs = [['W:0.5um - H:6um', 'W:0.75um - H:6um', 'W:1um - H:6um', 'W:1.25um - H:6um', 'W:1.5um - H:6um', 'W:1.75um - H:6um', 'W:2um - H:6um', 'W:3um - H:6um', 'W:4um - H:6um', 'W:6um - H:6um', 'W:8um - H:6um', 'W:10um - H:6um', 'W:14um - H:6um', 'W:18um - H:6um', 'W:22um - H:6um', 'W:26um - H:6um',],
    ['W:0.5um - H:24um', 'W:0.75um - H:24um', 'W:1um - H:24um', 'W:1.25um - H:24um', 'W:1.5um - H:24um', 'W:1.75um - H:24um', 'W:2um - H:24um', 'W:3um - H:24um', 'W:4um - H:24um', 'W:6um - H:24um', 'W:8um - H:24um', 'W:10um - H:24um', 'W:14um - H:24um', 'W:18um - H:24um', 'W:22um - H:24um', 'W:26um - H:24um',]]
    elif exp_name.startswith('Directionality constrain'):
        backw_designs = ['backward rescue loops',
                    'backward rescue loops angled',
                    'backward stomache',
                    'backward negative control']
        p = test_kruskal_wallis(exp_dat.loc[backw_designs], 'Backw.' + exp_name, passed_designs=None)
        forw_designs = ['forward rescue loops',
                    'forward rescue loops angled',
                    'forward stomache',
                    'forward negative control']
        p = test_kruskal_wallis(exp_dat.loc[forw_designs], 'Forw.' + exp_name, passed_designs=None)

    else:
        p = test_kruskal_wallis(exp_dat, exp_name, passed_designs=designs)
    
    


Index(['Cue gradient', 'Directionality constrain', 'Edge transition',
       'Radial detachment', 'Edge attachment', 'Joining channels',
       'Growth speed'],
      dtype='object', name='exp_name')
Cue gradient DIV:merge
Cue gradient DIV:merge: n levels:11:
 Kruskal Wallis t=45.779, p=0.0000
 --> ***


['480um x 12/12'], ['480um x 12/12']: p: 0.4952
['480um x 12/12'], ['480um x 12/11']: p: 0.3533
['480um x 12/12'], ['480um x 12/10']: p: 0.3507
['480um x 12/12'], ['480um x 12/9']: p: 0.0365
['480um x 12/12'], ['480um x 12/8']: p: 0.0000
['480um x 12/12'], ['480um x 12/7']: p: 0.0030
['480um x 12/12'], ['480um x 12/6']: p: 0.0006
['480um x 12/12'], ['480um x 12/5']: p: 0.0001
['480um x 12/12'], ['480um x 12/4']: p: 0.0237
['480um x 12/12'], ['480um x 12/3']: p: 0.1016
['480um x 12/12'], ['480um x 12/2']: p: 0.0003
['480um x 12/11'], ['480um x 12/12']: p: 0.3533
['480um x 12/11'], ['480um x 12/11']: p: 0.4814
['480um x 12/11'], ['480um x 12/10']: p: 0.3709
['480um x 12/11'], ['480um x 1

/tmp/ipykernel_49698/2392974008.py:20: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels((lbls))
/tmp/ipykernel_49698/2392974008.py:22: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels((lbls), rotation=45, ha='right')


Directionality constrain DIV:merge
Backw.Directionality constrain DIV:merge: n levels:4:
 Kruskal Wallis t=7.721, p=0.0522
 --> N.S.


['backward rescue loops'], ['backward rescue loops']: p: 0.4897
['backward rescue loops'], ['backward rescue loops angled']: p: 0.0827
['backward rescue loops'], ['backward stomache']: p: 0.0342
['backward rescue loops'], ['backward negative control']: p: 0.2620
['backward rescue loops angled'], ['backward rescue loops']: p: 0.0827
['backward rescue loops angled'], ['backward rescue loops angled']: p: 0.4885
['backward rescue loops angled'], ['backward stomache']: p: 0.1072
['backward rescue loops angled'], ['backward negative control']: p: 0.0169
['backward stomache'], ['backward rescue loops']: p: 0.0342
['backward stomache'], ['backward rescue loops angled']: p: 0.1072
['backward stomache'], ['backward stomache']: p: 0.4885
['backward stomache'], ['backward negative control']: p: 0.0263
['backward negative control'], ['backward rescue loops']: p: 0.2

Joining channels DIV:merge
Joining channels DIV:merge: n levels:8:
 Kruskal Wallis t=31.626, p=0.0000
 --> ***


['angled entry'], ['angled entry']: p: 0.4990
['angled entry'], ['5um opening']: p: 0.1587
['angled entry'], ['inlay']: p: 0.4699
['angled entry'], ['1.5um opening']: p: 0.0000
['angled entry'], ['3um opening']: p: 0.0051
['angled entry'], ['2x size']: p: 0.0011
['angled entry'], ['tangential entry']: p: 0.0003
['angled entry'], ['negative control']: p: 0.0042
['5um opening'], ['angled entry']: p: 0.1587
['5um opening'], ['5um opening']: p: 0.4981
['5um opening'], ['inlay']: p: 0.3007
['5um opening'], ['1.5um opening']: p: 0.0165
['5um opening'], ['3um opening']: p: 0.1617
['5um opening'], ['2x size']: p: 0.0939
['5um opening'], ['tangential entry']: p: 0.0067
['5um opening'], ['negative control']: p: 0.0843
['inlay'], ['angled entry']: p: 0.4699
['inlay'], ['5um opening']: p: 0.3007
['inlay'], ['inlay']: p: 0.4992
['inlay'], ['1.5um opening']: p: 0.0009
['inlay'], ['3um ope